In [1]:
# Imports
%gui tk
import tkinter as tk
import math
import time
import pandas as pd
from collections import defaultdict
import random

In [2]:
# Piece classes
class Fox:
    def __init__(self, board, row, col, radius_frac=0.35, color="red"):
        self.board = board
        self.row = row
        self.col = col
        self.color = color
        self.radius_frac = radius_frac
        self.item_id = self.board._draw_piece(self, color=self.color, radius_frac=self.radius_frac, owner="Fox")
        self.board._occupy(self, row, col)

    def move(self, direction): # direction = Up, Down, Left, or Right
        if not self.board._is_turn(self) or self.board.game_over:
            return False
        dr, dc = self.board._dir_delta(direction)
        if dr is None:
            return False
        nr, nc = self.row + dr, self.col + dc
        if not self.board._in_bounds(nr, nc) or not self.board._is_empty(nr, nc):
            return False
        self.board._relocate(self, nr, nc)
        if self.board._fox_on_far_side():
            self.board._end_game(winner="Fox")
            return True
        self.board._next_turn()
        return True


class Hound:
    def __init__(self, board, row, col, radius_frac=0.35, color="blue"):
        self.board = board
        self.row = row
        self.col = col
        self.color = color
        self.radius_frac = radius_frac
        self.item_id = self.board._draw_piece(self, color=self.color, radius_frac=self.radius_frac, owner="Hounds")
        self.board._occupy(self, row, col)

    def move(self, direction): # direction = Up, Down, Left, or Right
        if not self.board._is_turn(self) or self.board.game_over:
            return False
        if direction.lower() == "down":  # Hounds cannot move down
            return False
        dr, dc = self.board._dir_delta(direction)
        if dr is None:
            return False
        nr, nc = self.row + dr, self.col + dc
        if not self.board._in_bounds(nr, nc) or not self.board._is_empty(nr, nc):
            return False
        self.board._relocate(self, nr, nc)
        if not self.board._fox_has_moves():
            self.board._end_game(winner="Hounds")
            return True
        self.board._next_turn()
        return True


# Main Gameboard
class gameboard:
    MODES = ("Player", "FoxRandom", "HoundsRandom", "ShortestPath", "Mini-Max")

    def __init__(self, master, rows=6, cols=6, tile_size=60,
                 dark="#555555", light="#DDDDDD", outline="black"):
        self.master = master
        self.rows = rows
        self.cols = cols
        self.tile_size = tile_size
        self.dark = dark
        self.light = light
        self.outline = outline

        # geometry
        self.r = tile_size / math.sqrt(2)
        self.pad_x = self.r + 2
        self.pad_y = self.r + 2

        # canvas size
        self.width  = (cols + rows) * self.r + 2 * self.pad_x
        self.height = (cols + rows) * self.r + 2 * self.pad_y

        # UI container
        self.top = tk.Frame(master)
        self.top.pack()

        # Canvas
        self.canvas = tk.Canvas(self.top, width=self.width, height=self.height,
                                bg="white", highlightthickness=0)
        self.canvas.pack(pady=5)

        # Game state
        self.occ = {}
        self.item_to_piece = {}
        self.selected = None
        self.current_turn = "Fox"
        self.game_over = False
        self.winner = None
        self.fox_mode = "Player"
        self.hounds_mode = "Player"
        self.fox = None
        self.hounds = []

        self.canvas.bind("<Button-1>", self._on_click_canvas)
        self.master.bind("<Key>", self._on_keypress)

        self._draw_board()
        self._create_turn_hud("Waiting…")

    # Start and end games
    def game_start(self, fox_mode="Player", hounds_mode="Player"):
        self.fox_mode, self.hounds_mode = fox_mode, hounds_mode
        self._start_new_game()
        return self.winner  # returns after game ends (if blocking)

    def quit(self):
        try:
            self.master.destroy()
        except Exception:
            pass

    # Setup
    def _start_new_game(self):
        # Clear board
        self.canvas.delete("piece")
        self.canvas.delete("overlay")
        self.occ.clear()
        self.item_to_piece.clear()
        self.selected = None
        self.current_turn = "Fox"
        self.game_over = False
        self.winner = None

        # Create pieces
        self.fox = Fox(self, 0, 0)
        self.hounds = [
            Hound(self, 5, 5),
            Hound(self, 5, 3),
            Hound(self, 4, 4),
            Hound(self, 3, 5)
        ]

        self.canvas.tag_raise(self.fox.item_id)
        for h in self.hounds:
            self.canvas.tag_raise(h.item_id)

        self._update_turn_hud(self.current_turn)

        start_mode = self.fox_mode if self.current_turn == "Fox" else self.hounds_mode
        if start_mode == "FoxRandom":
            self.Fox_Random_AI()
        elif start_mode == "HoundsRandom":
            self.Hounds_Random_AI()
        elif start_mode == "ShortestPath":
            self.Fox_Short_Path_AI()
        elif start_mode == "Mini-Max":
            self.Hounds_Minimax_AI()

    # Drawing
    def _draw_board(self):
        self.canvas.delete("tile")
        for r in range(self.rows):
            for c in range(self.cols):
                cx, cy = self._tile_center(r, c)
                coords = [cx, cy - self.r, cx + self.r, cy, cx, cy + self.r, cx - self.r, cy]
                color = self.dark if (r + c) % 2 == 0 else self.light
                self.canvas.create_polygon(coords, fill=color, outline=self.outline, tags=("tile",))

    def _create_turn_hud(self, turn_text):
        self.canvas.delete("turnhud")
        pad = 10
        w, h = 120, 30
        self.canvas.create_rectangle(pad, pad, pad + w, pad + h,
                                     fill="#f4f4f4", outline="#555", tags=("turnhud",))
        self.canvas.create_text(pad + w / 2, pad + h / 2,
                                text=f"Turn: {turn_text}", font=("Arial", 10, "bold"),
                                tags=("turnhud",))

    def _update_turn_hud(self, turn_text):
        self._create_turn_hud(turn_text)

    # Geometry
    def _tile_center(self, r, c):
        return ((c - r) * self.r + (self.cols * self.r) + self.pad_x,
                (c + r) * self.r + self.pad_y)

    def _in_bounds(self, r, c):
        return 0 <= r < self.rows and 0 <= c < self.cols

    def _dir_delta(self, direction):
        d = direction.lower()
        return {
            "up": (-1, -1),
            "down": (1, 1),
            "left": (1, -1),
            "right": (-1, 1)
        }.get(d, (None, None))

    # Occupancy
    def _is_empty(self, r, c):
        return (r, c) not in self.occ

    def _occupy(self, piece, r, c):
        self.occ[(r, c)] = piece

    def _vacate(self, r, c):
        self.occ.pop((r, c), None)

    def _relocate(self, piece, nr, nc):
        self._vacate(piece.row, piece.col)
        piece.row, piece.col = nr, nc
        self._occupy(piece, nr, nc)
        self._position_piece(piece)

    # Turn conditions
    def _is_turn(self, piece):
        return (self.current_turn == "Fox" and isinstance(piece, Fox)) or \
               (self.current_turn == "Hounds" and isinstance(piece, Hound))

    def _next_turn(self):
        if self.game_over:
            return

        self.current_turn = "Hounds" if self.current_turn == "Fox" else "Fox"
        self._update_turn_hud(self.current_turn)

        # If Fox's turn begins and has no moves, Hounds instantly win.
        if self.current_turn == "Fox" and not self._fox_has_moves() and not self.game_over:
            self._end_game("Hounds")
            return

        mode = self.fox_mode if self.current_turn == "Fox" else self.hounds_mode
        if mode == "HoundsRandom":
            self.Hounds_Random_AI()
        elif mode == "FoxRandom":
            self.Fox_Random_AI()
        elif mode == "Mini-Max":
            self.Hounds_Minimax_AI()
        elif mode == "ShortestPath":
            self.Fox_Short_Path_AI()

    def _fox_has_moves(self):
        r, c = self.fox.row, self.fox.col
        for d in ("Up", "Down", "Left", "Right"):
            dr, dc = self._dir_delta(d)
            nr, nc = r + dr, c + dc
            if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                return True
        return False

    def _fox_on_far_side(self):
        return (self.fox.row + self.fox.col) == (self.rows + self.cols - 2)

    def _end_game(self, winner):
        self.game_over = True
        self.winner = winner
        self._dim_and_announce(winner)

    # Overlay
    def _dim_and_announce(self, winner):
        self.canvas.create_rectangle(0, 0, self.width, self.height,
                                     fill="black", stipple="gray50", outline="", tags=("overlay",))
        text, color = ("Win", "green") if winner == "Fox" else ("Lose", "red")
        self.canvas.create_text(self.width / 2, self.height / 2,
                                text=text, fill=color,
                                font=("Arial", int(self.tile_size * 1.2), "bold"),
                                tags=("overlay",))
        self.canvas.tag_raise("overlay")

    # Piece Draw/Move
    def _draw_piece(self, piece, color, radius_frac=0.35, outline="white", owner=""):
        cx, cy = self._tile_center(piece.row, piece.col)
        r = self.tile_size * radius_frac
        item = self.canvas.create_oval(cx - r, cy - r, cx + r, cy + r,
                                       fill=color, outline=outline, width=2, tags=("piece", owner))
        self.item_to_piece[item] = piece
        return item

    def _position_piece(self, piece):
        cx, cy = self._tile_center(piece.row, piece.col)
        r = self.tile_size * piece.radius_frac
        self.canvas.coords(piece.item_id, cx - r, cy - r, cx + r, cy + r)

    # Manual play: click + arrow
    def _on_click_canvas(self, event):
        if self.game_over:
            return
        mode = self.fox_mode if self.current_turn == "Fox" else self.hounds_mode
        if mode != "Player":
            return
        items = self.canvas.find_overlapping(event.x, event.y, event.x, event.y)
        for item in reversed(items):
            if item in self.item_to_piece:
                piece = self.item_to_piece[item]
                if self._is_turn(piece):
                    self._highlight_selection(piece)
                    break

    def _highlight_selection(self, piece):
        if self.selected:
            self.canvas.itemconfigure(self.selected.item_id, width=2)
        self.selected = piece
        if piece:
            self.canvas.itemconfigure(piece.item_id, width=4)

    def _on_keypress(self, event):
        if self.game_over or not self.selected:
            return
        key_map = {"Up": "Up", "Down": "Down", "Left": "Left", "Right": "Right"}
        if event.keysym not in key_map:
            return
        moved = self.selected.move(key_map[event.keysym])
        if moved:
            self.canvas.itemconfigure(self.selected.item_id, width=2)
            self.selected = None

    # AI Movement
    def Fox_Random_AI(self):
        """
        Pick a random legal move for whichever side’s turn it is (Fox or Hounds).
        If no moves are available, end the game accordingly.
        """
        if self.game_over:
            return

        # Pick the active side
        if self.current_turn == "Fox":
            movers = [self.fox]
        else:
            movers = self.hounds

        legal_moves = [] # (piece, direction)

        for piece in movers:
            # candidate directions that preserve dark tiles
            dirs = ["Up", "Down", "Left", "Right"]

            for d in dirs:

                dr, dc = self._dir_delta(d)
                if dr is None:
                    continue

                nr, nc = piece.row + dr, piece.col + dc
                if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                    legal_moves.append((piece, d))

        if not legal_moves:
            # no moves available means losing condition for the current side
            loser = self.current_turn
            self._end_game(winner="Hounds" if loser == "Fox" else "Fox")
            return

        # Choose and execute a random move
        piece, direction = random.choice(legal_moves)
        piece.move(direction)



    def Hounds_Random_AI(self):
        """
        Pick a random legal move for whichever side’s turn it is (Fox or Hounds).
        If no moves are available, end the game accordingly.
        """
        if self.game_over:
            return

        # Pick the active side
        if self.current_turn == "Fox":
            movers = [self.fox]
        else:
            movers = self.hounds

        legal_moves = [] # (piece, direction)

        for piece in movers:
            # candidate directions that preserve dark tiles
            dirs = ["Up", "Down", "Left", "Right"]

            for d in dirs:
                # Hounds cannot move Down
                if isinstance(piece, Hound) and d == "Down":
                    continue

                dr, dc = self._dir_delta(d)
                if dr is None:
                    continue

                nr, nc = piece.row + dr, piece.col + dc
                if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                    legal_moves.append((piece, d))

        if not legal_moves:
            # no moves available means losing condition for the current side
            loser = self.current_turn
            self._end_game(winner="Hounds" if loser == "Fox" else "Fox")
            return

        # Choose and execute a random move
        piece, direction = random.choice(legal_moves)
        piece.move(direction)


    def Hounds_Minimax_AI(self):
        pass

    def Fox_Short_Path_AI(self):
        pass
        

In [3]:
root = tk.Tk()
root.title("Fox and Hounds")
board = gameboard(root)
board.game_start("Player", "Player")

In [ ]:
# Config
N_GAMES = 100
CONDITIONS = [
    ("FoxRandom",  "HoundsRandom"),     # random-random
    ("FoxRandom",  "Mini-Max"),         # random-ai
    ("ShortestPath","HoundsRandom"),    # ai-random
    ("ShortestPath","Mini-Max"),        # ai-ai
]
MAX_STEPS = 5000 # safety cap for update cycles
SLEEP_SEC = 0.00005 # delay between GUI updates

# SimBoard counts moves
class SimBoard(gameboard):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.turn_count = 0

    def _relocate(self, piece, nr, nc):
        self.turn_count += 1
        return super()._relocate(piece, nr, nc)


def run_single_game(fox_mode, hounds_mode, gnum, max_steps=MAX_STEPS, sleep_sec=SLEEP_SEC):
    root = tk.Tk()
    root.withdraw()
    board = SimBoard(root)

    cond_name = f"{fox_mode.lower()}-{hounds_mode.lower()}".replace("mini-max", "ai")
    print(f"\nGame {gnum:03d} START — Condition: {cond_name}")

    board.game_start(fox_mode, hounds_mode)

    steps = 0
    while not board.game_over and steps < max_steps:
        root.update()
        time.sleep(sleep_sec)
        steps += 1

    if board.game_over:
        winner = board.winner
    else:
        winner = "Timeout"

    turns = getattr(board, "turn_count", None)
    print(f"Game {gnum:03d} END   — Winner: {winner}, Turns: {turns}")

    try:
        root.destroy()
    except Exception:
        pass

    return {"condition": cond_name, "winner": winner, "turns": turns}


def run_batch(n_games=N_GAMES, conditions=CONDITIONS):
    results = []
    gnum = 1
    for fox_mode, hounds_mode in conditions:
        for _ in range(n_games):
            res = run_single_game(fox_mode, hounds_mode, gnum)
            results.append(res)
            gnum += 1
    return pd.DataFrame(results)


# RUN
df_results = run_batch()

# SUMMARY
print("\n" + "="*70)
print("FINAL SUMMARY STATS")
print("="*70)

summary_counts = (
    df_results
    .groupby(["condition","winner"])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)
summary_turns = (
    df_results[df_results["winner"].isin(["Fox","Hounds"])]
    .groupby("condition")["turns"]
    .agg(["count","mean","median","min","max"])
    .sort_index()
)

print("\n-- Win/Loss Counts --")
print(summary_counts)

print("\n-- Turns Summary --")
print(summary_turns)

print("\nDone.")